In [2]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow.experimental.numpy as tnp
tf.enable_eager_execution()

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

FileNotFoundError: [Errno 2] No such file or directory: 'Lyrics_Counterparts.txt'

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [ ]:
tokenizer.texts_to_sequences(["First"])

In [ ]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [ ]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [6]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

NameError: name 'encoded' is not defined

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])

NameError: name 'batch_size' is not defined

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=5, callbacks=[ResetStatesCallback()])

In [14]:
model.save('stateful_rnn_lyrics')

In [15]:
tf.executing_eagerly()

True

In [16]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [17]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'u'

In [18]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new,steps=1)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [19]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char (text, temperature)
    return text

In [23]:
print(complete_text("a", temperature=1))

ains

this is the heart between nothing more can timely where i just from rorring to find you

any "r


In [24]:
print(complete_text("b", temperature=1))

beging
and we run from right among earth i am a god
caused me for right what i'll allow you i vill la


In [25]:
print(complete_text("f", temperature=1))

f than but i need souls of the fell that a dollection on our casualtyen
all i’vensle with from the pr
